我们建议将其`tf.keras`用作构建神经网络的高级API。也就是说，大多数TensorFlow API都可用于急切执行。

In [1]:
from __future__ import absolute_import,division,print_function,unicode_literals
import tensorflow as tf
tf.enable_eager_execution()

##### 图层：常用的有用操作集
大多数情况下，为机器学习模型编写代码时，您希望在比单个操作和单个变量操作更高的抽象级别上操作。

许多机器学习模型可以表达为相对简单的层的组合和堆叠，TensorFlow提供了一组许多常用层，以及您从头开始或作为组合形式编写您自己的应用程序特定层的简单方法。现有的图层。

TensorFlow包括完整Keras在tf.keras封装API，并建立自己的模型时，Keras层是非常有用的。

In [2]:
layer = tf.keras.layers.Dense(100)

layer = tf.keras.layers.Dense(10,input_shape=(None,5))

可以在文档中看到预先存在的图层的完整列表。它包括Dense（完全连接层），Conv2D，LSTM，BatchNormalization，Dropout等等。

In [3]:
layer(tf.zeros([10,5]))

<tf.Tensor: id=29, shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [4]:
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.02703106,  0.1097222 , -0.38245437,  0.33900416,  0.08631504,
          0.31437337, -0.14168555,  0.25208265,  0.40494126, -0.11019355],
        [ 0.5268741 ,  0.134467  ,  0.08782214, -0.16496739,  0.1699614 ,
         -0.4695339 , -0.16040725, -0.36093932,  0.08162427,  0.3598907 ],
        [ 0.35165733,  0.34063965,  0.20525932,  0.04473281,  0.2263971 ,
         -0.3793932 ,  0.5835542 , -0.3484208 ,  0.51846975, -0.45106402],
        [-0.19596559,  0.06160885,  0.5485396 ,  0.50070363,  0.10172641,
          0.03270119, -0.4327421 , -0.28987616, -0.44751507, -0.26784557],
        [ 0.18373251,  0.10383552,  0.30938315,  0.3378886 ,  0.3059216 ,
         -0.5948317 ,  0.04403949,  0.46445078, -0.131719  , -0.46110553]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [5]:
layer.kernel,layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.02703106,  0.1097222 , -0.38245437,  0.33900416,  0.08631504,
          0.31437337, -0.14168555,  0.25208265,  0.40494126, -0.11019355],
        [ 0.5268741 ,  0.134467  ,  0.08782214, -0.16496739,  0.1699614 ,
         -0.4695339 , -0.16040725, -0.36093932,  0.08162427,  0.3598907 ],
        [ 0.35165733,  0.34063965,  0.20525932,  0.04473281,  0.2263971 ,
         -0.3793932 ,  0.5835542 , -0.3484208 ,  0.51846975, -0.45106402],
        [-0.19596559,  0.06160885,  0.5485396 ,  0.50070363,  0.10172641,
          0.03270119, -0.4327421 , -0.28987616, -0.44751507, -0.26784557],
        [ 0.18373251,  0.10383552,  0.30938315,  0.3378886 ,  0.3059216 ,
         -0.5948317 ,  0.04403949,  0.46445078, -0.131719  , -0.46110553]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

#### 实现自定义图层
实现自己的层的最佳方法是扩展tf.keras.Layer类并实现：*` __init__`，您可以在其中执行所有与输入无关的初始化* `build`，您可以在其中了解输入张量的形状并可以执行其余的初始化* `call`，在哪里进行正向计算

请注意，您不必等到build调用创建变量，您也可以在其中创建变量`__init__`。但是，创建它们的优点`build`是它可以根据图层将要操作的输入的形状启用后期变量创建。另一方面，创建变量`__init__`意味着需要明确指定创建变量所需的形状。

In [6]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self,num_outputs):
        super(MyDenseLayer,self).__init__()
        self.num_outputs = num_outputs
        
    def build(self,input_shape):
        self.kernel = self.add_variable("kernel",
                                       shape=[int(input_shape[-1]),
                                             self.num_outputs])
        
    def call(self,input):
        return tf.matmul(input,self.kernel)
    
layer = MyDenseLayer(10)
print(layer(tf.zeros([10,5])))
print(layer.trainable_variables)

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(10, 10), dtype=float32)
[<tf.Variable 'my_dense_layer/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[ 0.20593005,  0.5688477 ,  0.1135866 , -0.2665477 , -0.08567792,
         0.10195535,  0.10942197,  0.17719942, -0.53376853,  0.59056836],
       [ 0.2534322 , -0.4339968 , -0.5301333 , -0.51648116, -0.5540666 ,
        -0.44647628, -0.34695452, -0.31965867, -0.06045049, -0.42835337],
       [-0.1425826 ,  0.43701023, -0.5482861 , -0.21374512, -0.00261331,
         0.02865189,  0.46238118, -0.12298393, -0.418347  ,  0.62997156],
       [-0.6033793 , -0.33934224,  0.04732275,  0.5452964 , -0.49277607,
        -0.303539  ,  0.0986321 ,  0

如果尽可能使用标准层，则整体代码更易于阅读和维护，因为其他读者将熟悉标准层的行为。如果你想使用tf.keras.layers或tf.contrib.layers中不存在的图层，请考虑提交github问题，或者更好的是向我们发送拉取请求！

#### 模型：组成图层
机器学习模型中许多有趣的层状事物是通过组合现有层来实现的。例如，resnet中的每个残余块是卷积，批量标准化和快捷方式的组合。

创建包含其他图层的类似图层的东西时使用的主类是tf.keras.Model。实现一个是通过继承自tf.keras.Model完成的。

In [7]:
class ResnetIdentityBlock(tf.keras.Model):
    def __init__(self,kernel_size,filters):
        super(ResnetIdentityBlock,self).__init__(name='')
        filters1,filters2,filters3 = filters
        
        self.conv2a = tf.keras.layers.Conv2D(filters1,(1,1))
        self.bn2a = tf.keras.layers.BatchNormalization()
        
        self.conv2b = tf.keras.layers.Conv2D(filters2,kernel_size,padding='same')
        self.bn2b = tf.keras.layers.BatchNormalization()
        
        self.conv2c = tf.keras.layers.Conv2D(filters3,(1,1))
        self.bn2c = tf.keras.layers.BatchNormalization()
        
    def call(self,input_tensor,training=False):
        x = self.conv2a(input_tensor)
        x = self.bn2a(x,training=training)
        x = tf.nn.relu(x)
        
        x = self.conv2b(x)
        x = self.bn2b(x,training=training)
        x = tf.nn.relu(x)
        
        x = self.conv2c(x)
        x = self.bn2c(x,training=training)
        
        x += input_tensor
        return tf.nn.relu(x)
    
block = ResnetIdentityBlock(1,[1,2,3])
print(block(tf.zeros([1,2,3,3])))
print([x.name for x in block.trainable_variables])

Instructions for updating:
Colocations handled automatically by placer.
tf.Tensor(
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], shape=(1, 2, 3, 3), dtype=float32)
['resnet_identity_block/conv2d/kernel:0', 'resnet_identity_block/conv2d/bias:0', 'resnet_identity_block/batch_normalization_v1/gamma:0', 'resnet_identity_block/batch_normalization_v1/beta:0', 'resnet_identity_block/conv2d_1/kernel:0', 'resnet_identity_block/conv2d_1/bias:0', 'resnet_identity_block/batch_normalization_v1_1/gamma:0', 'resnet_identity_block/batch_normalization_v1_1/beta:0', 'resnet_identity_block/conv2d_2/kernel:0', 'resnet_identity_block/conv2d_2/bias:0', 'resnet_identity_block/batch_normalization_v1_2/gamma:0', 'resnet_identity_block/batch_normalization_v1_2/beta:0']


然而，在很多时候，组成许多层的模型只是将一层接一层地称为一层。这可以使用`tf.keras.Sequential`在非常少的代码中完成

In [8]:
my_seq = tf.keras.Sequential([
    tf.keras.layers.Conv2D(1,(1,1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(2,1,padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(3,(1,1)),
    tf.keras.layers.BatchNormalization()
])
my_seq(tf.zeros([1,2,3,3]))

<tf.Tensor: id=509, shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>